In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [19]:
# 데이터셋으로 CIFAR-10 사용
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])

train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False, num_workers=2)

print('CIFAR-10 data ready!')

CIFAR-10 data ready!


In [20]:
# 인셉션 블록

class InceptionBlock(nn.Module):
    def __init__(self, in_channels, c1, c3r, c3, c5r, c5, pool_proj):
        super(InceptionBlock, self).__init__()
        # Branch1: 1x1 Conv
        self.branch1 = nn.Conv2d(in_channels, c1, kernel_size=1, bias=False)
        # Branch2: 1x1 -> 3x3
        self.branch2_1 = nn.Conv2d(in_channels, c3r, kernel_size=1, bias=False)
        self.branch2_2 = nn.Conv2d(c3r, c3, kernel_size=3, padding=1, bias=False)
        # Branch3: 1x1 -> 5x5
        self.branch3_1 = nn.Conv2d(in_channels, c5r, kernel_size=1, bias=False)
        self.branch3_2 = nn.Conv2d(c5r, c5, kernel_size=5, padding=2, bias=False)
        # Branch4: Pool -> 1x1
        self.pool = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.branch4 = nn.Conv2d(in_channels, pool_proj, kernel_size=1, bias=False)
        self.bn = nn.BatchNorm2d(c1 + c3 + c5 + pool_proj)

    def forward(self, x):
        b1 = self.branch1(x)

        b2 = self.branch2_1(x)
        b2 = F.relu(b2, inplace=True)
        b2 = self.branch2_2(b2)

        b3 = self.branch3_1(x)
        b3 = F.relu(b3, inplace=True)
        b3 = self.branch3_2(b3)

        b4 = self.pool(x)
        b4 = self.branch4(b4)

        out = torch.cat([b1, b2, b3, b4], dim=1)
        out = self.bn(out)

        return F.relu(out, inplace=True)

In [22]:
# -----------------------
# 4.3 Simple Inception Net for CIFAR-10
# -----------------------
class SimpleInceptionNet(nn.Module):
  def __init__(self, num_classes=10):
    super(SimpleInceptionNet, self).__init__()
    # initial conv
    self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(64)
    # Inception blocks
    self.inc1 = InceptionBlock(in_channels=64, c1=16, c3r=16, c3=24, c5r=16, c5=24, pool_proj=16)
    self.inc2 = InceptionBlock(in_channels=16+24+24+16, c1=32, c3r=32, c3=48, c5r=16, c5=24, pool_proj=24)
    # total out channels=128
    self.pool = nn.MaxPool2d(2,2)
    self.inc3 = InceptionBlock(in_channels=128, c1=32, c3r=32, c3=48, c5r=16, c5=24, pool_proj=24)
    self.bn2 = nn.BatchNorm2d(128)
    self.fc = nn.Linear(128, num_classes)

  def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = F.relu(x, inplace=True)
    x = self.inc1(x)
    x = self.inc2(x)
    x = self.pool(x)
    x = self.inc3(x)
    x = self.bn2(x)
    x = F.relu(x, inplace=True)
    x = F.adaptive_avg_pool2d(x, (1,1))
    x = x.view(x.size(0), -1)
    x = self.fc(x)
    return x

model = SimpleInceptionNet().to(device)
print(model)

SimpleInceptionNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (inc1): InceptionBlock(
    (branch1): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (branch2_1): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (branch2_2): Conv2d(16, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (branch3_1): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (branch3_2): Conv2d(16, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
    (pool): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
    (branch4): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (inc2): InceptionBlock(
    (branch1): Conv2d(80, 32, kernel_size=(1, 1), stride=(1, 1), bia

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
num_epochs = 5

for epoch in range(1, num_epochs+1):
  model.train()
  running_loss = 0.0
  for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()

avg_loss = running_loss / len(train_loader)
model.eval()
correct = 0
total = 0
with torch.no_grad():
  for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)
    out = model(images)
    _, predicted = torch.max(out, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

acc = 100.0 * correct / total
print(f"Epoch {epoch}, Loss: {avg_loss:.4f}, Test Acc: {acc:.2f}%")